<a href="https://colab.research.google.com/github/hhhhh0102/ner_project/blob/main/ner_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 모듈들을 설치합니다.
!pip install transformers
!pip install sentencepiece

In [ ]:
# 필요한 모듈 설치를 위해 github의 프로젝트를 마운트합니다.
!git clone https://github.com/hhhhh0102/ner_project.git

In [ ]:
# 모델을 불러오기 위해 구글드라이브를 마운트합니다.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 마운트된 구글드라이브 경로를 추가해줍니다.
import os
import sys
sys.path.append("/content/ner_project/ner_model")

In [ ]:
import numpy as np
import tensorflow as tf
from Preprocess import preprocess
from make_model import make_model
from tokenizer import KoBertTokenizer

# 한글 텍스트 파일의 경로를 input으로 받아 해당 파일 텍스트 값들의 NER값을 예측한 텍스트 파일을 만드는 함수입니다.
def ner_task(textpath):
    # 한글 텍스트 토크나이저를 불러옵니다.
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

    # 예측 모델을 불러옵니다.
    model = tf.keras.models.load_model("/content/drive/MyDrive/my_model.h5")

    # 텍스트 파일을 예측 모델의 input을 만들기 위한 초기 설정을 합니다
    my_model = make_model()
    my_model.model_input.set_dataset("/content/ner_project/ner_model/train_data")
    my_model.model_input.make_input()
   
    line_list = []

    # 한글 텍스트 파일을 읽고 문장들을 line_list변수에 할당합니다.
    f = open(textpath, mode='r', encoding='utf-8')
    while True:
        line = f.readline()
        if not line: break
        line_list.append(line)
    f.close()

    # 한글 텍스트들을 예측 모델을 사용하여 NER값들을 예측하고, 예측 값들을 포맷에 맞게 ner_result.txt파일에 저장합니다.
    f = open('ner_result.txt', mode='wt', encoding='utf-8')
    for l in list(range(len(line_list))):

        tokenized_sentence = np.array([tokenizer.encode(line_list[l], max_length=88, pad_to_max_length=True)])
        tokenized_mask = np.array([[int(x != 1) for x in tokenized_sentence[0].tolist()]])
        ans = model.predict([tokenized_sentence, tokenized_mask])
        ans = np.argmax(ans, axis=2)

        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, ans[0]):

            if (token.startswith("▁")):
                new_labels.append(my_model.model_input.pr.index_to_ner[label_idx])
                new_tokens.append(token[1:])
            elif (token == '[CLS]'):
                pass
            elif (token == '[SEP]'):
                pass
            elif (token == '[PAD]'):
                pass
            elif (token != '[CLS]' or token != '[SEP]'):
                new_tokens[-1] = new_tokens[-1] + token

        for ll in list(range(len(new_tokens))):
            f.write("{}\t{}\t{}\n".format(ll + 1, new_tokens[ll], new_labels[ll]))
        f.write("\n")

    f.close()

In [ ]:
# pre_trained_data.txt : NER 정답지가 있는 한글 텍스트 파일
ner_task("/content/ner_project/pre_trained_data.txt")